Кобзарь О.С. Водопьян А.О. Хабибуллин Р.А. 5.08.2019

# Применение уравнение материального баланса к залежам УВ



In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uTools.data_workflow as data_workflow
import uniflocpy.uReservoir.MatBalance as MB_module
import uniflocpy.uTools.uconst as uc
import uniflocpy.uPVT.PVT_fluids as PVT

import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt


# Истощение пласта до давления насыщения

Уравнение материального баланса для данного случая будет выглядеть следующим образом

$$N_p B_o + W_p B_w = N B_{oi} (\frac {B_o - B_{oi} } {B_{oi} \Delta p }   + \frac{c_w S_{wc} + c_f}{1 - S_{wc}}) \Delta p  $$

т.е. для такого режима работа залежи свойственны следующие соотношения:

$$ m = 0 $$
$$ W_e = 0 $$
$$ R_s = R_{si} = R_p $$

Также в примере существует упрощение - отсутствие добычи воды:

$$ W_p = 0 $$

Далее с учетом того, что коэффициент сжимаемости нефти

$$ c_o = \frac {B_o - B_{oi} } {B_{oi} \Delta p } $$

И система является двухфазной:

$$ S_o = 1 - S_{wc} $$

Уравнение можно упростить до

$$N_p B_o + W_p B_w = N B_{oi} c_{eff} \Delta p  $$

В котором эффективная сжимаемость недонасыщенной системы:

$$ c_{eff} = \frac{c_o S_o +  c_w S_{wc} + c_f}{1 - S_{wc}} $$

При данных упращениях КИН 

$$ КИН = \frac {N}{N_P}$$

Может быть выражен следующим  образом:

$$ КИН = \frac {B_{oi} c_{eff} \Delta p }{B_o}$$


In [ ]:

c_oil_1bar = 14.5 * 10 ** (-5)
c_wat_1bar = 4.35 * 10 ** (-5)
c_res_1bar = 7.25 * 10 ** (-5)

p_res_init_bar = 250
p_drop_bar = 103
t_res_init_c = 80
t_res_c = 80
S_wat_connate_d = 0.25
p_reservoir_bar = pbcal_bar = p_res_init_bar - p_drop_bar


In [ ]:
MB_ex1  = MB_module.MatBalance()

In [ ]:
MB_ex1.c_oil_1bar = c_oil_1bar
MB_ex1.c_wat_1bar = c_wat_1bar
MB_ex1.c_res_1bar = c_res_1bar

MB_ex1.t_reservoir_c = t_res_c
MB_ex1.t_reservoir_init_c = t_res_init_c
MB_ex1.p_reservoir_init_bar = p_res_init_bar
MB_ex1.S_wat_connate_d = S_wat_connate_d
MB_ex1.fluid.pbcal_bar = pbcal_bar


In [ ]:
MB_ex1.calc_depletion_above_pb(p_reservoir_bar)

In [ ]:
MB_ex1.oil_recovery_perc

# Режим истощения нефтяной  залежи (замкнутый пласт)

Если предположить, что скважина работает с постоянным дебитом в замкнутом пласте - на контуре питания происходит падение давления. Когда пластовое давление выше давление насыщения, газ растворен в нефти, проявляется упругий режим. Когда давление падает ниже давления насыщения, газ выделяется из нефти в отдельную фазу, и расширяясь, начинает более активно поддерживать пластовое давление. 

Уравнение материального баланса, для данного случая (чисто нефтяная залежь) выражается следующим образом

$$ N_p [B_o + (Rp - R_s) B_g] +  N[(B_o - B_oi) + (R_{si} - R_s)B_g] + \frac{ N B_{oi} (c_w S_{wc} + c_f) \Delta p }  { 1 - S_{wc} } $$


# Поиск среднего пластового давления после эксплуатации



In [ ]:
p_res_init_bar = 250
t_res_init_c = 80
r_drainage_m = 250
porosity_d = 0.25
q_oil_surf_m3day = 50
h_eff_res_m = 8
c_system_1bar = 7.5 * 10 ** (-5)
t_end_year = 1
t_step_days = 30.33
S_wat_connate_d = 0.25

fluid = PVT.FluidStanding()
fluid.pbcal_bar = 100
fluid.rsb_m3m3 = 100
fluid.calc(p_res_init_bar, t_res_init_c)


STOIIP_by_VOL_m3 = uc.pi * r_drainage_m ** 2 * h_eff_res_m * porosity_d * (1 - S_wat_connate_d) / fluid.b_oil_m3m3

N_cum_oil_recovery_m3 = q_oil_surf_m3day * t_step_days

MB = MB_module.MatBalance()
MB_data = data_workflow.Data()
fluid_data = data_workflow.Data()

MB.fluid = fluid
MB.rp_m3m3 = MB.fluid.rs_m3m3

MB.STOIIP_by_VOL_m3 = STOIIP_by_VOL_m3
c_wat_1bar = 4.35 * 10 ** (-5)
c_res_1bar = 7.25 * 10 ** (-5)

MB.c_wat_1bar = c_wat_1bar
MB.c_res_1bar = c_res_1bar

MB.t_reservoir_init_c = t_res_init_c
MB.p_reservoir_init_bar = p_res_init_bar
MB.S_wat_connate_d = S_wat_connate_d

for i in range(1, 50):
    N_cum_oil_recovery_m3 = q_oil_surf_m3day * t_step_days * i
    MB.calc_depletion_above_and_below_pb(N_cum_oil_recovery_m3)
    MB_data.get_data(MB)
    fluid_data.get_data(MB.fluid)


In [ ]:
MB_data.print_all_names()

In [ ]:
fluid_data.print_all_names()

In [ ]:
def trace(data_x, data_y, namexy):
    tracep = go.Scattergl(
        x = data_x,
        y = data_y,
        name = namexy,
        mode = 'lines'
    )
    return tracep

def plot():
    layout = dict(title = 'Расчет материального баланса')

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')
    
def create_traces_list_by_num(data_x_values, data_y, num_y_list):
    trace_list = []
    for i in num_y_list:
        namexy = data_y.get_name(i)
        this_trace = trace(data_x_values, data_y.get_values(i), namexy)
        trace_list.append(this_trace)
    return trace_list

def connect_traces(traces1, trace2):
    connected_traces = []
    for i in traces1:
        connected_traces.append(i)
    for j in trace2:
        connected_traces.append(j)
    return connected_traces

In [ ]:
x_data_month = np.asarray(MB_data.get_values(5)) / (q_oil_surf_m3day * t_step_days)
#x_data_month = np.asarray(MB_data.get_values(5))

In [ ]:
numbers_fluid_list = [16,26,3]
data_trace_fluid = create_traces_list_by_num(x_data_month, fluid_data,  numbers_fluid_list)
numbers_MB_list = [3]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = (connect_traces(data_trace_fluid, data_trace_MB))
plot()

In [ ]:
numbers_MB_list = [5,10]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = data_trace_MB
plot()

In [ ]:
numbers_MB_list = [8, 21,25]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = data_trace_MB
plot()

In [ ]:
numbers_MB_list = [28]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = data_trace_MB
plot()

In [ ]:
numbers_MB_list = [18,19,20,22,23,24]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = data_trace_MB
plot()

In [ ]:
numbers_MB_list = [29,30,31,32,33,34]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = data_trace_MB
plot()